## Evaluation

Batch prediction and evaluation are very similar. They are based on DataFlow pipeline and CloudML provides Evaluate and Prediction DataFlow transform. Datalab can generate DataFlow pipeline code template for you, just like Preprocessing.

Run "%mlalpha evaluate" to generate input cell. After fill in the required fields, it becomes:
```
%%mlalpha evaluate
preprocessed_eval_data_path: /content/datalab/tmp/ml/census/preprocessed/features_eval*
metadata_path: /content/datalab/tmp/ml/census/preprocessed/metadata.yaml
model_dir: /content/datalab/tmp/ml/census/model/model
output_dir: /content/datalab/tmp/ml/census/evaluate
```

In [8]:

# header
"""
Following code is generated from command line:
%%mlalpha evaluate
preprocessed_eval_data_path: /content/datalab/tmp/ml/census/preprocessed/features_eval*
metadata_path: /content/datalab/tmp/ml/census/preprocessed/metadata.yaml
model_dir: /content/datalab/tmp/ml/census/model/model
output_dir: /content/datalab/tmp/ml/census/evaluate

Please modify as appropriate!!!
"""

# imports
import apache_beam as beam
import google.cloud.ml as ml
import google.cloud.ml.analysis as analysis
import google.cloud.ml.io as io
import json
import os

# defines
def extract_values((example, prediction)):
  import tensorflow as tf
  tf_example = tf.train.Example()
  tf_example.ParseFromString(example.values()[0])
  feature_map = tf_example.features.feature
  values = {'target': feature_map['target'].float_list.value[0]}
  values.update(prediction)
  return values

OUTPUT_DIR = '/content/datalab/tmp/ml/census/evaluate'
pipeline = beam.Pipeline('DirectPipelineRunner')


# evaluation

eval_features = (pipeline | 'ReadEval' >> io.LoadFeatures('/content/datalab/tmp/ml/census/preprocessed/features_eval*'))
trained_model = pipeline | 'LoadModel' >> io.LoadModel('/content/datalab/tmp/ml/census/model/model')
evaluations = (eval_features | 'Evaluate' >> ml.Evaluate(trained_model) |
    beam.Map('ExtractEvaluationResults', extract_values))
eval_data_sink = beam.io.TextFileSink(os.path.join(OUTPUT_DIR, 'eval'), shard_name_template='')
evaluations | beam.io.textio.WriteToText(os.path.join(OUTPUT_DIR, 'eval'), shard_name_template='')

# analysis

# run pipeline
pipeline.run()


View the output eval file:

In [9]:
!head /content/datalab/tmp/ml/census/evaluate/eval

{u'score': [62744.8359375], 'target': 172300.0, u'key': [172300.0]}
{u'score': [62741.1328125], 'target': 28000.0, u'key': [28000.0]}
{u'score': [54228.66015625], 'target': 37300.0, u'key': [37300.0]}
{u'score': [47622.3984375], 'target': 42400.0, u'key': [42400.0]}
{u'score': [61382.60546875], 'target': 100000.0, u'key': [100000.0]}
{u'score': [58404.56640625], 'target': 53000.0, u'key': [53000.0]}
{u'score': [60505.54296875], 'target': 63300.0, u'key': [63300.0]}
{u'score': [64454.1484375], 'target': 56200.0, u'key': [56200.0]}
{u'score': [64122.703125], 'target': 132000.0, u'key': [132000.0]}
{u'score': [53015.11328125], 'target': 55000.0, u'key': [55000.0]}


To generate a pipeline that runs in cloud, simply add --cloud to "%mlalpha evaluate". Also all paths need to be GCS paths.

In [4]:
import os

bucket = 'gs://' + datalab_project_id() + '-sampledata'
eval_data_path = os.path.join(bucket, 'census', 'preprocessed', 'features_eval')
metadata_path = os.path.join(bucket, 'census', 'preprocessed', 'metadata.yaml')
model_path = os.path.join(bucket, 'census', 'trained', 'model')
output_dir = os.path.join(bucket, 'census', 'evaluate')
eval_file = os.path.join(output_dir, 'eval*')

In [ ]:
%%ml evaluate --cloud
preprocessed_eval_data_path: $eval_data_path
metadata_path: $metadata_path
model_dir: $model_path
output_dir: $output_dir

After you run the generated code, you can go to Developer Console to see the DataFlow job: https://pantheon.corp.google.com/dataflow (and select the right project). Also you can check the results as below:

In [6]:
!gsutil cat $eval_file | head -5

{u'score': [62152.2109375], 'target': 172300.0, u'key': ['705']}
{u'score': [50811.1875], 'target': 28000.0, u'key': ['1204']}
{u'score': [41454.01953125], 'target': 37300.0, u'key': ['1758']}
{u'score': [70818.65625], 'target': 42400.0, u'key': ['2534']}
{u'score': [63665.48046875], 'target': 100000.0, u'key': ['3172']}
